In [34]:
import pandas as pd

df = pd.read_csv('data.csv')

In [35]:
# Remove the Timestamp column
df = df.drop(columns=['Timestamp'])

In [36]:
# Rename the columns
df = df.rename(columns={
    "Thank you for your participation. If you want to receive the survey results, enter your email address here. We won't share your email address with anyone, nor send you any ads.": 'email',
    "What's your age?": 'age',
    "What's your gender?": 'gender',
    "Where do you live?": 'homeland',
    "Where do you usually enjoy your holidays? Select all that apply.": 'target',
    "When on vacation, where do you usually stay? Select all that apply.": 'accommodation',
    "Who do you usually enjoy your holidays with? Select all that apply.": 'company',
    "What activities best describe your typical summer vacation? Select all that apply.": 'activities',
    "How do you choose where to go on vacation? Select all that apply.": 'decision',
    "What means of transport do you normally take when going on vacation? Select all that apply.": 'transport',
    "How much did you spend in total during your last summer holidays? Use this format: <cost> <currency> (e.g. 350 €)": 'cost',
    "How much did you enjoy your last summer vacation?": 'enjoyment',
    "Thank you for your participation. If you want to receive the survey results, enter your email address here. We won't share your email address with anyone, nor send you any ads.": 'email',
})

In [37]:
# Extract the email column into a separate array
emails = df['email'].values
# Remove the email column from the dataframe
df = df.drop(columns=['email'])

In [38]:
# Change all the homeland values that contain 'europe' to 'europe'
df['homeland'] = df['homeland'].apply(lambda x: 'europe' if 'europe' in x.lower() else x)

In [39]:
# Change all the gender values that aren't 'Male' or 'Female' into 'Other'
df['gender'] = df['gender'].apply(lambda x: 'Other' if x != 'Male' and x != 'Female' else x)

In [53]:
# Parse the raw cost values and convert to euros

import re


def assume_eur_if_no_symbol(string):
    try:
        return int(string)
    except ValueError:
        return None


def find_number(string) -> int | None:
    numbers = re.findall(r'\d+', string)

    if len(numbers) == 0:
        return None

    return max([int(x) for x in numbers])


def parse_cost(string: str) -> int | None:
    eur_symbols = ['€', 'eur', 'euro', 'euros']
    usd_symbols = ['$', 'usd', 'dollar', 'dollars']
    gbp_symbols = ['£', 'gbp', 'pound', 'pounds']
    cad_symbols = ['cad']
    aud_symbols = ['aud']
    all_symbols = eur_symbols + usd_symbols + gbp_symbols + cad_symbols + aud_symbols

    string = str(string).lower()

    assume_eur = assume_eur_if_no_symbol(string)

    # Branch based on currency
    if any([x in string for x in eur_symbols]):
        return find_number(string)
    
    elif assume_eur is not None:
        return assume_eur

    elif any([x in string for x in usd_symbols]):
        n = find_number(string)
        if n is None:
            return None
        return int(n * 1.00)
    
    elif any([x in string for x in gbp_symbols]):
        n = find_number(string)
        if n is None:
            return None
        return int(n * 1.15)

    elif any([x in string for x in cad_symbols]):
        n = find_number(string)
        if n is None:
            return None
        return int(n * 0.76)
    
    elif any([x in string for x in aud_symbols]):
        n = find_number(string)
        if n is None:
            return None
        return int(n * 0.67)


df['cost'] = df['cost'].apply(parse_cost)


In [56]:
df['cost'].head(20)

0      100.0
1      750.0
2      234.0
3        NaN
4        NaN
5     8000.0
6        3.0
7      400.0
8        NaN
9        NaN
10       NaN
11    1000.0
12    5000.0
13    2000.0
14    1437.0
15    1500.0
16     800.0
17       NaN
18     684.0
19     919.0
Name: cost, dtype: float64